In [41]:
from hedypet.utils import DERIVATIVES_ROOT, load_splits, get_time_frames_midpoint
from nifti_dynamic.utils import load_tac
from nifti_dynamic.patlak import roi_patlak
from parse import parse
from tqdm import tqdm
import warnings
import pandas as pd 


In [125]:
subs = load_splits()["all"]
subs.remove("sub-017")


import json
def load_json(file_path):
    with open(file_path,"r") as handle:
        d = json.load(handle)
    return d

region_names = {
    'ts_total' :load_json("/homes/hinge/Projects/hedyPET/src/hedypet/analysis/seg-total-classes.json"),
    'synthseg' :load_json("/homes/hinge/Projects/hedyPET/src/hedypet/analysis/seg-synthseg_classes.json"),
    'synthsegparc' : load_json("/homes/hinge/Projects/hedyPET/src/hedypet/analysis/seg-synthseg_classes.json"),
    'ts_tissue' :load_json("/homes/hinge/Projects/hedyPET/src/hedypet/analysis/seg-tissue-classes.json"),
    'ts_body' : {"1":"trunk","2":"extremeties"},
    'totalimage' : {"1":"body"},
}



In [127]:
df.task.unique()

array(['ts_total', 'ts_tissue', 'ts_body', 'synthseg', 'synthsegparc',
       'totalimage'], dtype=object)

In [130]:
tacs_if

[PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-0/tac_1'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-0/tac_2'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-0/tac_3'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-0/tac_4'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-1/tac_1'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-1/tac_2'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-1/tac_3'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortasegments/erosion-1/tac_4'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortavois_ml-1_width-3/erosion-0/tac_1'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-099/acdynPSF/aortavois_ml-1_width-3/erosion-0/tac_2'),
 P

In [128]:

warnings.filterwarnings("ignore")
frames = [2,3,4,5,6,7,8,9,10,11,12]

ki_data = []

for sub in tqdm(subs):
    #Find all tacs
    tacs_root = (DERIVATIVES_ROOT / f"tacs/{sub}/acdynPSF")
    tacs = list(tacs_root.glob("**/tac*"))
    tacs_if = [x for x in tacs if "aorta" in str(x)]
    tacs_organs = [x for x in tacs if x not in tacs_if]
    t_middle = get_time_frames_midpoint(sub)

    # Try all combinations of inputs functions and num_frames
    for tac_if_path in tacs_if:
        tac_if, _, _ = load_tac(tac_if_path)
        for tac_organ_path in tacs_organs:
            tac_organ, _, n = load_tac(tac_organ_path)
            for frame in frames:
                slope, intercept, X, Y = roi_patlak(tac_organ,tac_if,t_middle,frame)

                tags_if = parse('{}/tacs/{}/acdynPSF/{task}/erosion-{erosion}/tac_{ix}',str(tac_if_path)).named
                tags_organ = parse('{}/tacs/{sub}/acdynPSF/{task}/erosion-{erosion}/tac_{ix}',str(tac_organ_path)).named

                combined_tag_if = tags_if["task"]+"_"+tags_if["erosion"] + "_" + tags_if["ix"]
                
                series = {"slope":float(slope),"n":int(n[0]),"tac_if_path":tac_if_path,"tac_organ_path":tac_organ_path,"frames":frame}
                series["if_tag"] = combined_tag_if
                series.update(tags_organ)
                series["region"] = region_names[series["task"]][series["ix"]]
                ki_data.append(series)

df = pd.DataFrame(ki_data)
df.to_pickle("patlak_ki.pkl")

100%|██████████| 99/99 [3:05:00<00:00, 112.13s/it]  
